# Connecting to Foursquare and Yelp APIs 
For each of the bike stations in Part 1, Foursquare and Yelp APIs used to retrieve information for the selected POIs:

Selected POIs are including following groups:
* Entertainment (Museum, Cinema, ...) 
* Education (School, University, College, ...) 
* Dining (Restuerant, Coffee shop, Fastfood, ...) 
* Park 
* Shopping (Shopping mall, store, grocery, ...)
* Metro_station
* Library
* Financial institution (Bank, ATM, Credit Union)


In [2]:
# imports
import pandas as pd
import os
import requests
import json
from pandas import json_normalize

import sys
# API Keys stored in 'D:/0-LHL-Activities/API_KEY/my_api_keys.py' and are not inserted here for security
sys.path.append('D:/0-LHL-Activities/API_KEY/')
import my_api_keys

# Check API Keys
#print(my_api_keys.api_key('FOURSQUARE_API_KEY'))
#print(my_api_keys.api_key('YELP_API_KEY'))

# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [26]:
# Function get_venues_4square sends request to FourSquare
def get_venues_4square( categories , latitude , longitude , radius , limit):
    
    # Make url
    url = "https://api.foursquare.com/v3/places/search?categories={}&ll={},{}&radius={}&limit={}".format(
                categories,
                latitude,
                longitude,
                radius,
                limit)

    # Create dictionary for headers
    headers = {"Accept": "application/json"}
    headers['Authorization'] = my_api_keys.api_key('FOURSQUARE_API_KEY')

    # Send Request
    result = requests.get(url, headers=headers)
    
    # Compute number of places
    d=result.json()
    number_of_places = len(json_normalize(d['results']))

    return number_of_places


Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [ ]:
# This code sends request to FourSquare and gets number of places for different categories and bike stations 

# Read api_key from OS environment variable
api_key    = os.environ['FOURSQUARE_API_KEY']
place_radius = 1000
place_limit = 50

# Load CityBikes data, saved in CSV file
df_citybikes = pd.read_csv("data/city_bikes.csv" , encoding = "unicode_escape")

# Point of interest (POI), based on FourSquare categories ( https://location.foursquare.com/places/docs/categories )
# List of selected categories
poi_name=[ 'entertainment' , 'education' , 'dining' , 'park' ,'shopping','metro_station'                         ,'library'     ,'bank','atm','creditunion' ]
poi_catid=[ '10001,10002,10004' , '12009,12013' , '13071,13072,13035,13145' , '16032,16036','17114,17115','19046','12080,12031','11046','11044','11045']

#Initializing foursquare dataframe
header = ['uid'] + poi_name
lst = len( df_citybikes ) * [ len( header ) * [0] ]
df_4square = pd.DataFrame( lst , columns = header , dtype = int )    

# Request Number Of All Place Category For All CityBikes Latitudes and Longitudes
for i in range(len( df_citybikes )):
    df_4square['uid'][i] = df_citybikes['uid'][i]   
    for j in range(len(poi_name)):
        df_4square[poi_name[j]][i] = get_venues_4square(poi_catid[j] , 
                                                        df_citybikes['latitude'][i] , 
                                                        df_citybikes['longitude'][i] , 
                                                        place_radius , 
                                                        place_limit)
       


Put your parsed results into a DataFrame

In [2]:
# Save results to CSV file
df_4square.to_csv('data/FourSquare.csv' , sep = ',' , encoding = 'utf-8',index=False)

# Show result
df_4square

,uid,entertainment,education,dining,park,shopping,metro_station,library,bank,atm,creditunion
0,66,33,50,50,8,42,4,45,50,13,50
1,26,4,50,46,5,12,0,10,50,3,50
2,20,13,50,46,7,5,0,6,50,2,50
3,21,20,50,50,6,5,0,8,50,3,50
4,22,18,50,50,6,10,0,8,50,3,50
...,...,...,...,...,...,...,...,...,...,...,...
302,344,20,50,50,4,14,4,10,50,4,50
303,136,11,50,50,20,15,0,16,50,12,50
304,5,30,50,50,7,45,4,49,50,13,50
305,346,7,50,21,7,3,0,6,50,5,50


# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [35]:
# Function get_venues_yelp sends request to Yelp
def get_venues_yelp( categories , latitude , longitude , radius , limit):
    
    # Make url
    url = "https://api.yelp.com/v3/businesses/search?latitude={}&longitude={}&term={}&radius={}&sort_by=best_match&limit={}".format(
                    latitude,
                    longitude,
                    categories,
                    radius,
                    limit)
   
    # Create dictionary for headers
    headers = {
    "accept": "application/json",
    "Authorization": "Bearer "+ my_api_keys.api_key('YELP_API_KEY') }

    # Send Request
    result = requests.get(url , headers = headers)
    
    # Determine number of places
    d=result.json()
    number_of_places = d['total']

    return number_of_places

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [ ]:
# This code sends request to Yelp and gets number of places for different categories and bike stations 

# Read api_key from OS environment variable
api_key    = os.environ['YELP_API_KEY']
place_radius = 1000
place_limit = 50

# Load CityBikes data, saved in CSV file
df_citybikes = pd.read_csv("data/city_bikes.csv",encoding="unicode_escape")

# Point of interest (POI)
# List of selected categories
poi_name=['entertainment','education','dining','park','shopping','metro_station','library','financial']

#Initializing foursquare dataframe
header =['uid']+poi_name
lst = len(df_citybikes)*[len(header)*[0]]
df_yelp = pd.DataFrame(lst, columns =header, dtype = int)    

# Request Number Of All Place Category For All CityBikes Latitudes and Longitudes
for i in range(len(df_citybikes)):
    df_yelp['uid'][i]=df_citybikes['uid'][i]   
    for j in range(len(poi_name)):
        df_yelp[poi_name[j]][i]=get_venues_yelp(poi_name[j],
                                                df_citybikes['latitude'][i],
                                                df_citybikes['longitude'][i],
                                                place_radius,
                                                place_limit)
       


Put your parsed results into a DataFrame

In [3]:
# Save results to CSV file
df_yelp.to_csv('data/Yelp.csv' , sep = ',' , encoding = 'utf-8',index=False)

# Show result
df_yelp 

,uid,entertainment,education,dining,park,shopping,metro_station,library,financial
0,66,17,16,10,3,78,3,3,3
1,26,1,6,6,0,53,0,0,1
2,20,2,3,5,1,26,0,0,0
3,21,2,5,5,0,44,0,0,0
4,22,3,6,7,1,45,0,1,0
...,...,...,...,...,...,...,...,...,...
302,344,5,9,21,0,84,0,0,5
303,136,2,6,6,4,26,0,0,0
304,5,17,17,12,1,80,3,3,4
305,346,0,3,1,3,9,0,0,0


# Comparing Results

##### Which API provided you with more complete data? Provide an explanation. 
According to FourSquare and Yelp descriptions and details, FourSquare data is categorized into many major and minor categories, but Yelp provides them in large categories. 
On the other side, the number of venues is limited to 50 by FourSquare. So FourSquare does not provide the correct value for high among.
So I decided to use Yelp information.

# Get the top 10 restaurants according to their rating
In this section restaurants and rates are retrieved from Yelp and the top 10 restaurants according to their rating are listed.

In [3]:
# This code gets the top 10 restaurants according to their rating in Yelp

# Function get_venues_yelp_rate sends request to Yelp and returns restaurants info including rate
def get_venues_yelp_rate( categories , latitude , longitude , radius , limit):

    # Make url
    url = "https://api.yelp.com/v3/businesses/search?latitude={}&longitude={}&term={}&radius={}&sort_by=best_match&limit={}".format(
                    latitude,
                    longitude,
                    categories,
                    radius,
                    limit)
   
    # Create dictionary for headers
    headers = {
    "accept": "application/json",
    "Authorization": "Bearer "+api_key}

    # Send Request
    result = requests.get(url , headers = headers)
    return result.json()

#======================================================

api_key    = my_api_keys.api_key('YELP_API_KEY')
place_radius = 1000
place_limit = 50

# Load CityBikes data, saved in CSV file
df_citybikes = pd.read_csv("data/citybikes.csv",encoding="unicode_escape")

# list of categories for listing rate
poi_name=['dining']

lst=[]

# Request Name, Rate and Number of Reviews of category 'dining' For All CityBikes Latitudes and Longitudes
for i in range(len(df_citybikes)):
    for j in range(len(poi_name)):
        d=get_venues_yelp_rate('dining',
                        df_citybikes['latitude'][i],
                        df_citybikes['longitude'][i],
                        place_radius,
                        place_limit)
        dd=d['businesses']

        for i in dd:
            address=i['location']['display_address']

            full_address=''
            for j in address: 
                full_address=full_address+j+','

            full_address=full_address[0:len(full_address)-1]

            new_rest_info={'name':i['name'],
                           'full_address':full_address,
                           'rate':i['rating'],
                           'reviews':i['review_count']}
            # Add new_rest_info to list if not added before
            if (new_rest_info not in lst):
                lst=lst+[new_rest_info]

# Convert to dataframe
df= pd.DataFrame(lst)

# sort rate in descending order
df.sort_values(by=['rate','reviews'], inplace=True, ascending=[False,False])


In [4]:
# Save results to CSV file
df.to_csv('data/Yelp - Rates.csv' , sep = ',' , encoding = 'utf-8',index=False)

# Show result
df.head(10)
max_rate=(df['rate']).max()
print('Total number of restaurants with rating ',max_rate,' is ' ,(df['rate']==max_rate ).sum(),'. So list is sorted by rate and number of reviews.')
df.head(10)

Total number of resturants with rating  5.0  is  10 . So list is sorted by rate and number of reviews.


,name,full_address,rate,reviews
14,Bondinho Pão de Açúcar,"Praça General Tibúrcio, s/n,Rio de Janeiro - R...",5.0,137
18,Oteque,"R. Conde de Irajá, 581,Rio de Janeiro - RJ 222...",5.0,5
23,L'atelier du Cuisinier,"R. Teófilo Otoni, 97,Rio de Janeiro - RJ 20090...",5.0,5
48,Antiquarius,"R. Aristides Espinola, 19,Rio de Janeiro - RJ ...",5.0,3
63,Arena Ipanema Hotel,"R. Francisco Otaviano, 131,Rio de Janeiro - RJ...",5.0,3
19,Tortin,"R. da Matriz, 54,Rio de Janeiro - RJ 22260-100...",5.0,1
96,Explorer Bar,"R. Almirante Alexandrino, 399,Santa Teresa,Rio...",5.0,1
106,Hostel Cristo Rei,"R. Cosme Velho, 596,Rio de Janeiro - RJ 22241-...",5.0,1
118,SUD - O Pássaro Verde Café,"R. Visc. de Carandaí, 35,Jardim Botânico,Rio d...",5.0,1
130,Companhia do Garfo,"Av. Princesa Isabel, 214,Rio de Janeiro - RJ 2...",5.0,1
